In [21]:
## 광물 데이터 분석
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

import pandas as pd

In [22]:
## 사용할 데이터 불러오기
df = pd.read_csv('./data/sonar3.csv', header= None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


In [23]:
## 광물이 몇개인지 확인
df[60].value_counts()

60
1    111
0     97
Name: count, dtype: int64

In [24]:
## 특성과 정답 분리
X = df.iloc[:,0:60]
y = df.iloc[:,60]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.3, shuffle= True)

In [26]:
## 모델 객체 선언
model = Sequential()
## 3개의 layer생성
## 1. 노드 : 24개, input데이터 : 60개, 활성화 함수 : 렐루함수
## 2. 노드 : 10개, 활성화 함수 : 렐루함수
## 3. 노드 : 1개, 활성화 함수 : 시그모이드 함수
model.add(Dense(24, input_dim = 60, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [27]:
## 모델 컴파일
## 손실 함수 : binary_crossentropy, 옵티마이저 : adam
model.compile(loss='binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])

In [28]:
# ## 모델 훈련
# history = model.fit(X,y,epochs=200,batch_size=10)

In [29]:
## 모델 훈련
history = model.fit(X_train,y_train,epochs=200,batch_size=10)

Epoch 1/200
15/15 [==============================] - 1s 2ms/step - loss: 0.6871 - accuracy: 0.5241
Epoch 2/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6733 - accuracy: 0.6000
Epoch 3/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6645 - accuracy: 0.6621
Epoch 4/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6552 - accuracy: 0.6759
Epoch 5/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6482 - accuracy: 0.6345
Epoch 6/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6395 - accuracy: 0.6897
Epoch 7/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6306 - accuracy: 0.6897
Epoch 8/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6214 - accuracy: 0.6966
Epoch 9/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6143 - accuracy: 0.6483
Epoch 10/200
15/15 [==============================] - 0s 2ms/step - loss: 0.6081 - accuracy: 0.7379
Epoch 11/

In [30]:
## 모델 테스트 세트에 적용
score = model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

2/2 [==============================] - 0s 5ms/step - loss: 0.3650 - accuracy: 0.8730
Test accuracy:  0.8730158805847168


In [31]:
## 모델 저장
model.save('./data/model/my_model.hdf5')
## 모델 삭제
del model

In [32]:
model = load_model('./data/model/my_model.hdf5')

In [33]:
## 모델 테스트 세트에 적용
score = model.evaluate(X_test, y_test)
print('Test accuracy: ', score[1])

2/2 [==============================] - 0s 8ms/step - loss: 0.3650 - accuracy: 0.8730
Test accuracy:  0.8730158805847168


In [34]:
## 적은 데이터를 보완하기 위해 K교차 검증 사용
from sklearn.model_selection import KFold

In [35]:
k = 5
kfold = KFold(n_splits= k, shuffle= True)

# for train_index, test_index in kfold.split(X):
#     print(train_index)
#     print('\n')
#     print(test_index)


In [36]:
def model_func():
    model = Sequential()
    model.add(Dense(24, input_dim = 60, activation = 'relu'))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    return model

In [38]:
acc_score = []

for train_index, test_index in kfold.split(X):
    X_train = X.iloc[train_index,:]
    X_test = X.iloc[test_index,:]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
    
    model = model_func()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    hist = model.fit(X_train, y_train, epochs= 200, batch_size= 10, verbose= 0)
    score = model.evaluate(X_test, y_test)[1]
    acc_score.append(score)

2/2 [==============================] - 0s 8ms/step - loss: 0.2147 - accuracy: 0.8780


In [39]:
avg_score = sum(acc_score) / k

print('정확도: ', acc_score)
print('정확도 평균: ', avg_score)

정확도:  [0.761904776096344, 0.8333333134651184, 0.7142857313156128, 0.8536585569381714, 0.8780487775802612]
정확도 평균:  0.8082462310791015
